In [13]:
import sys  
sys.path.insert(0, r'C:\Users\monac\Documents\GitHub\RSA_Investigations')

In [14]:
from inspector import Metric
from inspector.utility import plot_distmat

from netrep.metrics.stochastic import EnergyStochasticMetric
from netrep.multiset import pairwise_distances

import numpy as np
import sklearn.datasets

In [15]:
class SyntheticNetworks():
    def __init__(self) -> None:
        pass

    def generate(self, networks = 3, n_samples = 500, n_classes = 4, n_features = 2):
        # seed = 42069
        # rng = np.random.default_rng(seed)

        X , y = [], []

        for i in range(networks):
            # Generate a new dataset
            _X, _y = sklearn.datasets.make_classification(
                n_samples=n_samples, 
                n_features=n_features,
                n_informative=n_features, # Meaning: all features are informative (no redundant or repeated features)
                n_redundant=0, # Meaning: no redundant features
                n_repeated=0, # Meaning: no repeated features
                n_classes=n_classes,
                n_clusters_per_class=1, # Meaning: each class is a single cluster
                # random_state=seed,
            )
            X.append(_X)
            y.append(_y)

        return X, y

In [16]:
# create 10 synthetic networks
network_generator = SyntheticNetworks()
networks_X, networks_y = network_generator.generate(networks=10)

# create an RSA object
netrep_metric = ESM()
ingestion_report = netrep_metric.ingest(networks_X, networks_y)
digestion_report = netrep_metric.digest()
energy_distance = netrep_metric.score(verbose=True)

netrep_metric.plot()

NameError: name 'ESM' is not defined

In [17]:
network_generator = SyntheticNetworks()
networks_X, networks_y = network_generator.generate(networks=3, n_samples=600, n_classes=4, n_features=2)

In [18]:
X1, Y1 = networks_X[0], networks_y[0]

In [19]:
X1.shape, Y1.shape

((600, 2), (600,))

In [20]:
import numpy as np

def slice_min_samples_by_class(X, Y, n_classes, n_min_samples, n_features):
    X_reshaped = np.zeros((n_classes, n_min_samples, n_features))
    Y_reshaped = np.zeros((n_classes, n_min_samples))

    for i in range(n_classes):
        class_indices = np.where(Y == i)[0] # Get the indices of samples belonging to class i
        np.random.shuffle(class_indices) # Shuffle the indices to get random samples
        X_class = X[class_indices[:n_min_samples]] # Get the first n_min_samples samples belonging to class i
        Y_class = Y[class_indices[:n_min_samples]]
        X_reshaped[i] = X_class.reshape(n_min_samples, -1)  # Reshape the samples into 2D array
        Y_reshaped[i] = Y_class

    return X_reshaped, Y_reshaped

In [21]:
def reshape_networks_by_class(X_all, Y_all, n_classes):
    '''Returns list of Networks in shape [(class, sample, feature), (class, sample, feature), ...]'''

    X_all_by_class = []
    Y_all_by_class = []

    for i in range(len(X_all)):
        X = X_all[i]
        Y = Y_all[i]

        X_by_class = [X[Y==i] for i in range(n_classes)]
        Y_by_class = [Y[Y==i] for i in range(n_classes)]

        X_all_by_class.append(X_by_class)
        Y_all_by_class.append(Y_by_class)

    return X_all_by_class, Y_all_by_class

In [22]:
networks_X[0].shape, len(networks_X)

((600, 2), 3)

In [23]:
X_all_by_class, Y_all_by_class = reshape_networks_by_class(networks_X, networks_y, 4)

In [24]:
len(X_all_by_class), len(X_all_by_class[0]), len(X_all_by_class[0][1]), len(X_all_by_class[0][0][0])

(3, 4, 151, 2)

In [25]:
def get_min_samples_per_class(X_all_by_class):
    '''Works accross all networks and returns the minimum number of samples per class'''

    minimum_samples_per_class = float('inf')

    for net_id, network in enumerate(X_all_by_class):
        
        for class_id, classes in enumerate(network):
            sample_length = len(classes)

            print(f"for network {net_id}| class {class_id} | found:  {sample_length} samples")

            if sample_length < minimum_samples_per_class:
                minimum_samples_per_class = sample_length

    return minimum_samples_per_class


In [26]:
n_min_samples = get_min_samples_per_class(X_all_by_class)

for network 0| class 0 | found:  150 samples
for network 0| class 1 | found:  151 samples
for network 0| class 2 | found:  149 samples
for network 0| class 3 | found:  150 samples
for network 1| class 0 | found:  150 samples
for network 1| class 1 | found:  150 samples
for network 1| class 2 | found:  150 samples
for network 1| class 3 | found:  150 samples
for network 2| class 0 | found:  148 samples
for network 2| class 1 | found:  150 samples
for network 2| class 2 | found:  150 samples
for network 2| class 3 | found:  152 samples


In [40]:
def slice_by_samples(X_all_by_class, n_min_samples):
    '''Returns a list of networks in shape np.array([(class, n_minsamples, feature), (class, n_minsamples, feature), ...])'''

    X_all_sliced = []

    for network in X_all_by_class:
        net_t = []

        for classes in network:
            net_t.append(classes[:n_min_samples])

        X_all_sliced.append(np.array(net_t))

    return X_all_sliced

In [41]:
X_all_reshaped = slice_by_samples(X_all_by_class, n_min_samples)

In [42]:
X_all_reshaped.shape

AttributeError: 'list' object has no attribute 'shape'

In [43]:
from inspector.rsa import NetRep

In [44]:
rep = NetRep()

In [45]:
rep.score(networks=X_all_reshaped)

array([[0.        , 0.27189048, 0.22689774],
       [0.27189048, 0.        , 0.20134913],
       [0.22689774, 0.20134913, 0.        ]])

In [ ]:
import numpy as np

# assume X_all_by_class is a list of lists with different lengths
# reshape the data to create a new 2D array with a homogeneous shape
X_reshaped = np.array([x[:10] for x in X_all_by_class])

# print the shape of the new array
print(X_reshaped.shape) # output: (3, 10)